# Entrenamiento Personalizado: StyleGan2-ADA

En este cuaderno, realizaremos transfer learning con StyleGAN2 y conjuntos de datos personalizados.

Esto significa que no entrenaremos la red GAN desde cero con nuestras imágenes (ya que toma alrededor de dos semanas), sino que usaremos el modelo ya entrenado en otras imágenes como punto de partida. Esto reducirá el tiempo de entrenamiento a unas horas al omitir las etapas iniciales donde la red neuronal aprende características de bajo nivel de imágenes que son muy similares para cualquier tipo de imágenes.

In [ ]:
#@title 1. Conectar Colab a  Google Drive
#@markdown Accede a tu Google Drive para cargar tu conjunto de datos, editar las imágenes y guardar los resultados.

from google.colab import drive
drive.mount('/content/drive')

In [1]:
#@title 2. Preparar Dataset: Recorte, Redimensionamiento y Compresión ZIP
#@markdown Permite recortar y preparar imágenes cuadradas para StyleGAN2, y luego comprimir la carpeta final en un ZIP listo para el entrenamiento.

import os, glob, zipfile, traceback
from math import ceil, floor
from PIL import Image
import numpy as np
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output

# ---------------------------
# 🔹 Funciones auxiliares
# ---------------------------
def crop_center(img, ratio=1.0):
    w, h = img.size
    min_side = min(w, h)
    crop_size = int(min_side * ratio)
    left = (w - crop_size) // 2
    top = (h - crop_size) // 2
    right = left + crop_size
    bottom = top + crop_size
    return img.crop((left, top, right, bottom))

def save_crop(img, count, out_dir, augment=False):
    img.save(f'{out_dir}{count:08d}.png')
    count += 1
    if augment:
        for angle in [90, 180, 270]:
            img_rot = img.rotate(angle, expand=True)
            img_rot.save(f'{out_dir}{count:08d}.png')
            count += 1
    return count

# ---------------------------
# 🔹 Widgets de interfaz
# ---------------------------
input_dir_widget = widgets.Text(
    value="/content/drive/MyDrive/Dataset/original",
    description="Origen:",
    layout=widgets.Layout(width='95%')
)

output_dir_widget = widgets.Text(
    value="/content/drive/MyDrive/Dataset/verificado",
    description="Salida:",
    layout=widgets.Layout(width='95%')
)

resize_widget = widgets.Dropdown(
    options=[256, 512, 1024],
    value=512,
    description="Tamaño:",
    layout=widgets.Layout(width='45%')
)

ratio_widget = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=1.0,
    step=0.05,
    description="🔳 Ratio:",
    continuous_update=True,
    layout=widgets.Layout(width='45%')
)

augment_widget = widgets.Checkbox(
    value=False,
    description="Aumentar dataset con rotaciones",
    layout=widgets.Layout(width='60%')
)

process_button = widgets.Button(
    description="🚀 Procesar imágenes",
    button_style='success',
    layout=widgets.Layout(width='60%', margin='10px 0px 10px 0px')
)

zip_button = widgets.Button(
    description="📦 Comprimir carpeta en ZIP",
    button_style='info',
    layout=widgets.Layout(width='60%', margin='10px 0px 10px 0px')
)

progress_label = widgets.Label(value="")
progress_bar = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, layout=widgets.Layout(width='90%'))
error_output = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='10px'))
output_area = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', padding='10px'))

# ---------------------------
# 🔹 Función principal de procesamiento
# ---------------------------
def process_dataset(b):
    clear_output(wait=True)
    display(ui)

    input_dir = input_dir_widget.value.strip()
    out_dir = output_dir_widget.value.strip()
    resize = int(resize_widget.value)
    ratio = ratio_widget.value
    augment = augment_widget.value

    if not out_dir.endswith("/"):
        out_dir += "/"

    # ❌ Verificar si hay espacios en la ruta
    if " " in out_dir or " " in input_dir:
        with output_area:
            clear_output(wait=True)
            print("⚠️ Error: No se permiten espacios en la ruta de origen o salida. Renombra las carpetas y vuelve a intentar.")
        return

    os.makedirs(out_dir, exist_ok=True)

    images = glob.glob(f'{input_dir}/*.tif') + \
             glob.glob(f'{input_dir}/*.png') + \
             glob.glob(f'{input_dir}/*.jpg') + \
             glob.glob(f'{input_dir}/*.jpeg') + \
             glob.glob(f'{input_dir}/*.bmp')

    total_images = len(images)
    cnt = 0
    failed = []

    with output_area:
        clear_output(wait=True)
        if total_images == 0:
            print("⚠️ No se encontraron imágenes en la carpeta de origen.")
            return
        print(f'📂 Procesando {total_images} imágenes...\n')

    progress_bar.value = 0
    progress_label.value = f"0 / {total_images} procesadas"

    for i, img_path in enumerate(images):
        try:
            img = Image.open(img_path)
            img = crop_center(img, ratio)
            img = img.resize((resize, resize), Image.Resampling.LANCZOS)

            if img.mode in ('RGBA', 'LA') or (img.mode == 'P' and 'transparency' in img.info):
                bg = Image.new('RGB', img.size, (255, 255, 255))
                bg.paste(img, (0, 0), img)
                img = bg
            elif img.mode != 'RGB':
                img = img.convert('RGB')

            cnt = save_crop(img, cnt, out_dir, augment)
        except Exception as e:
            failed.append((os.path.basename(img_path), str(e)))

        progress_bar.value = (i + 1) / total_images
        progress_label.value = f"{i+1} / {total_images} procesadas ({(i+1)/total_images*100:.1f}%)"

    with output_area:
        print(f'\n✅ Procesamiento completado.')
        print(f'🖼️ Total de imágenes guardadas: {cnt}')
        print(f'📁 Directorio de salida: {out_dir}')

    with error_output:
        clear_output(wait=True)
        if failed:
            print(f"\n⚠️ {len(failed)} imágenes no pudieron procesarse:\n")
            for name, err in failed:
                print(f"   • {name} → {err}")
        else:
            print("✅ Todas las imágenes fueron procesadas correctamente.")

# ---------------------------
# 🔹 Función de compresión ZIP
# ---------------------------
def compress_dataset(b):
    clear_output(wait=True)
    display(ui)

    out_dir = output_dir_widget.value.strip()
    if not out_dir.endswith("/"):
        out_dir += "/"

    if " " in out_dir:
        with output_area:
            clear_output(wait=True)
            print("⚠️ Error: No se permiten espacios en la ruta del dataset. Renombra la carpeta y vuelve a intentar.")
        return

    if not os.path.exists(out_dir):
        with output_area:
            clear_output(wait=True)
            print("⚠️ La carpeta de salida no existe. Procesa las imágenes primero.")
        return

    zip_path = out_dir.rstrip("/") + ".zip"
    files = glob.glob(os.path.join(out_dir, "*.*"))
    total_files = len(files)

    if total_files == 0:
        with output_area:
            clear_output(wait=True)
            print("⚠️ No hay imágenes para comprimir.")
        return

    with output_area:
        clear_output(wait=True)
        print(f"📦 Comprimiendo {total_files} archivos en: {zip_path}\n")

    progress_bar.value = 0
    progress_label.value = "0% completado"

    try:
        with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
            for i, file in enumerate(files):
                arcname = os.path.basename(file)
                zipf.write(file, arcname)
                progress_bar.value = (i + 1) / total_files
                progress_label.value = f"{(i + 1) / total_files * 100:.1f}% completado"

        with output_area:
            print(f"\n✅ Carpeta comprimida correctamente.")
            print(f"📁 Archivo ZIP creado: {zip_path}")
    except Exception as e:
        with error_output:
            clear_output(wait=True)
            print(f"⚠️ Error al comprimir: {str(e)}")

# ---------------------------
# 🔹 Eventos de botones
# ---------------------------
process_button.on_click(process_dataset)
zip_button.on_click(compress_dataset)

# ---------------------------
# 🔹 Interfaz principal
# ---------------------------
ui = widgets.VBox([
    widgets.HTML("<h3>🧩 Preparar Dataset: Recorte, Redimensionamiento y Compresión ZIP</h3>"),
    input_dir_widget,
    output_dir_widget,
    widgets.HBox([resize_widget, ratio_widget]),
    augment_widget,
    process_button,
    zip_button,
    progress_bar,
    progress_label,
    output_area,
    error_output
])

display(ui)


In [ ]:
#@title 3. Instalación de Librerías
#@markdown StyleGAN2-ADA se instalará en tu Google Drive para acelerar el proceso de entrenamiento.

#@markdown Ejecuta esta celda. Si ya has instalado el repositorio, omitirá el proceso de instalación y actualizará el directorio del repositorio. Si no lo has instalado, instalará todos los archivos necesarios.
#@markdown Puedes notar algunos errores: Ignóralos por ahora, son problemas de compatibilidad que no afectan nuestro trabajo.
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/angelv-salazar/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/angelv-salazar/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip uninstall jax jaxlib -y
!pip install "jax[cuda11_cudnn805]==0.3.10" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install torch==1.8.1 torchvision==0.9.1
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install scipy==1.10.1
!pip install ninja

%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

In [ ]:
#@title 4. Proceso de entrenamiento de Modelo StyleGAN2 Personalizado

#@markdown Ingresa la ruta del archivo *zip* del *dataset* creado en la celda anterior.
dataset = "/content/drive/MyDrive/data/gan01/verify.zip" #@param {type: "string"}

#@markdown Para el aprendizaje por transferencia, establece los puntos de partida **ffhq512** o **ffhq1024** según la resolución de tus imágenes. Esto debes hacerlo solo la primera vez.
#@markdown Si deseas reanudar el proceso de entrenamiento, omite el archivo zip y proporciona la ruta a tu último archivo *.pkl* que se encuentra en la ruta: **colab-sg2-ada-pytorch/stylegan-ada-pytorch/results** de tu Drive.
resume_from = "ffhq512" #@param {type: "string"}

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap=1 --cfg='11gb-gpu' --data={dataset} --aug=noaug --mirror=False --mirrory=False --metrics=None --resume={resume_from}

### Mientras estás entrenando...
¡Una vez que la celda anterior esté en ejecución, debería estar entrenando!

¡No cierres esta pestaña! Colab debe estar abierta y operativa para continuar con el entrenamiento.

Cada 40 minutos aproximadamente, se debería agregar una nueva línea a tu salida, indicando que aún estás entrenando. Dependiendo de la configuración de snapshot_count, deberías ver que la carpeta de resultados (/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results) en tu carpeta de Google Drive se llena tanto con muestras (fakesXXXXXx.jpg) como con pesos del modelo (model weights) (network-snapshot-XXXXXX.pkl). Vale la pena revisar las muestras durante el entrenamiento, pero no te preocupes demasiado por cada muestra individual.

Una vez que Colab se haya apagado (esto es frecuente debido a sus limites), puedes volver a reanudar el entrenamiento. Para esto debes ejecutar de nuevo algunas celdas del cuaderno: Montar Drive, Instalar Librerias y actualizar la ruta de tu último archivo .pkl en *resume_from* para continuar el entrenamiento desde el modelo más reciente.